# Cobordisms

## We define the main objects

In [ ]:
from klrw.perfect_complex import KLRWHomOfPerfectComplexes
from klrw.combinatorial_ebranes import CombinatorialEBrane
folder_path = "./pickles/"

We will need to 
- change the ground ring to a field [rational numbers],
- turn off all gradings except the equivariant one,
- set all auxilliary parameters to 1.

In [ ]:
def rational_base(E):
    cls, args, kwrds = E.klrw_algebra[E.k]._reduction
    R = QQ
    kwrds["base_R"] = R
    kwrds['vertex_scaling'] = False
    kwrds['edge_scaling'] = False
    kwrds["default_vertex_parameter"] = R.one()
    kwrds["default_edge_parameter"] = R.one()
    return cls(*args, **kwrds)

This defines an E brain after brading.

In [ ]:
def E_twisted(n, k, braid):
    E = CombinatorialEBrane(n, k)
    E.apply_braid(braid)
    E_complex = E.complex(*range(k))
    return E_complex.base_change(rational_base(E))

Here are the [twisted] E-branes that we will use:

In [ ]:
n, k = 4, 2
E1 = E_twisted(n, k, [])
E2 = E_twisted(n, k, [2,1])
Ep = E_twisted(n, k, [2,])
En = E_twisted(n, k, [-2,])

En

We will often compute exts between complexes of KRLW projectives [e.g. E-branes]

In [ ]:
def find_exts(domain, codomain, rang=None, max_eq_deg = 10, min_eq_deg=-10):
    exts ={}
    ext_dims = {}

    if rang is None:
        from itertools import product
        max_hom_deg = max(-deg1 + deg2 for deg1 in domain.projectives for deg2 in codomain.projectives)
        min_hom_deg = min(-deg1 + deg2 for deg1 in domain.projectives for deg2 in codomain.projectives)
        rang = product(range(min_hom_deg, max_hom_deg+1), range(min_eq_deg, max_eq_deg+1))

    for shift, i in rang:
        print("Working on {} {}".format(shift, i))
        rhom = KLRWHomOfPerfectComplexes(domain, codomain[shift,i])
        ext = rhom.ext(0)
        dim = len(ext.basis())
        if dim:
            exts[shift, i] = ext
            ext_dims[shift, i] = dim

    return exts, ext_dims

For future reference, let's find the dimensions of self-exts of an E-brane.
<br>
Here we do it for E1, but since brading is abn autoequivalence, it is the same if we replace E1 with any twisted E-brane.

In [ ]:
# for faster run can comment
exts, ext_dims = find_exts(E1, E1)
ext_dims

---

## Maps from elementary cobordism

Let's look at exts E1->E2.

In [ ]:
exts_E1_E2, ext_dims_E1_E2 = find_exts(E1, E2)
# for faster run: use
# exts_E1_E2, ext_dims_E1_E2 = find_exts(E1, E2, rang=((-1,0),))
ext_dims_E1_E2

We see dimensions (-1,0) and (-3,-2).
<br>
We expect (-1,0) to correspond to the "simplest" saddle-like cobordism.
<br>
(-3,-2) might come from attaching a handle to this cobordism.
<br>
[attaching handlebodies is the source of "dots" on cobordisms for Khovanov]

---

Let's look at exts E2->E1.

In [ ]:
exts_E2_E1, ext_dims_E2_E1 = find_exts(E2, E1)
# for faster run: use
# exts_E2_E1, ext_dims_E2_E1 = find_exts(E2, E1, rang=((-1,-2),))
ext_dims_E2_E1

We see dimensions (-1,-2) and (-3,-4).
<br>
We expect (-1,-2) to correspond to the "simplest" saddle-like cobordism.

---

## Cones

Now we look at cones over our candidates for the cobordisms.

In [ ]:
cone_E2_E1 = exts_E2_E1[-1,-2].basis()[0].cone()
cone_E1_E2 = exts_E1_E2[-1,0].basis()[0].cone()

We have a guess from Khovanov that cone_E2_E1 is En with some degree shift.
<br>
A guess for this degree shift comes from looking at Ext(cone_E2_E1,En) and comparing it to Ext(En,En)=Ext(E1,E1).

In [ ]:
# can comment for a faster run
find_exts(cone_E2_E1, En)

Note: this is the cohomology of the the unknot with shift (-1,-2).
<br>
So, we expect cone_E2_E1 to be En[-1,-2]


Let's check it.


We will find non-nil-homotopic maps
<br>
u: En[-1,-2] -> cone_E2_E1
<br>
and
<br>
v: cone_E2_E1 -> En[-1,-2].


Then we compute both their compositions and check that they are homotopic to a scalar.
<br>
Hence u and v are isomorphisms in the homotopy category
<br>
[so, also isomorphisms in the derived caterogy]

In [ ]:
En_shifted = En[-1, -2]

ext = KLRWHomOfPerfectComplexes(cone_E2_E1, En_shifted).ext(0)
assert len(ext.basis()) == 1, len(ext.basis())
v = ext.basis()[0]

ext = KLRWHomOfPerfectComplexes(En_shifted, cone_E2_E1).ext(0)
assert len(ext.basis()) == 1, len(ext.basis())
u = ext.basis()[0]

rhom_En_shifted = KLRWHomOfPerfectComplexes(En_shifted, En_shifted)
ext_En_shifted = rhom_En_shifted.ext(0)
hom_En_shifted = ext_En_shifted.hom_of_graded

print("v*u ~ scalar:", ext_En_shifted.if_homotopic_to_scalar(v*u))

rhom_cone_shifted = KLRWHomOfPerfectComplexes(cone_E2_E1, cone_E2_E1)
ext_cone_shifted = rhom_cone_shifted.ext(0)
hom_cone_shifted = ext_cone_shifted.hom_of_graded

print("u*v ~ scalar:", ext_cone_shifted.if_homotopic_to_scalar(u*v))

As expected, En[-1,-2] ~ cone_E2_E1!

---



Now, let's look at the other cobordism.

In [ ]:
# can comment for a faster run
find_exts(cone_E1_E2, Ep)

Note: this is the cohomology of the the unknot with shift (-1,0).
<br>
So, we expect cone_E1_E2 to be Ep[-1,0]


Let's check it.


We will find non-nil-homotopic maps
<br>
u: Ep[-1,0] -> cone_E1_E2
<br>
and
<br>
v: cone_E1_E2 -> Ep[-1,0].


Then we compute both their compositions and check that they are homotopic to a scalar.
<br>
Hence u and v are isomorphisms in the homotopy category
<br>
[so, also isomorphisms in the derived caterogy]

In [ ]:
Ep_shifted = Ep[-1, 0]

ext = KLRWHomOfPerfectComplexes(cone_E1_E2, Ep_shifted).ext(0)
assert len(ext.basis()) == 1, len(ext.basis())
v = ext.basis()[0]

ext = KLRWHomOfPerfectComplexes(Ep_shifted, cone_E1_E2).ext(0)
assert len(ext.basis()) == 1, len(ext.basis())
u = ext.basis()[0]

rhom_Ep_shifted = KLRWHomOfPerfectComplexes(Ep_shifted, Ep_shifted)
ext_Ep_shifted = rhom_Ep_shifted.ext(0)
hom_Ep_shifted = ext_Ep_shifted.hom_of_graded

print("v*u ~ scalar:", ext_Ep_shifted.if_homotopic_to_scalar(v*u))

rhom_cone_shifted = KLRWHomOfPerfectComplexes(cone_E1_E2, cone_E1_E2)
ext_cone_shifted = rhom_cone_shifted.ext(0)
hom_cone_shifted = ext_cone_shifted.hom_of_graded

print("u*v ~ scalar:", ext_cone_shifted.if_homotopic_to_scalar(u*v))

Cobordisms work as expected!